# Vannevar Labs Dataset for NatSec Hackathon 2024

Thanks for participating in the hackathon! This notebook is available at https://vl-nat-sec-hackathon-may-2024.s3.us-east-2.amazonaws.com/vl-data-download.ipynb, and we will be updating it over the course of the week with additional data and resources.

The dataset we are providing are of Russian social media posts from Telegram and VK that are related to current geopolitical events, a lot of them specifically about events occurring in Ukraine. The data in `attachment_urls` are media files that we will be providing in the same s3 bucket later this week.

If there are any issues with data access please email charu@vannevarlabs.com

In [27]:
!pip install boto3 botocore pandas

In [1]:
import boto3
import pandas as pd
from io import BytesIO
from botocore import UNSIGNED
from botocore.config import Config


# # Create a boto3 session with an anonymous user
# s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# bucket_name = 'vl-nat-sec-hackathon-may-2024'
# file_key = 'russia_social_media.csv'

# # Get the object from S3
# response = s3.get_object(Bucket=bucket_name, Key=file_key)

# file_content = response['Body'].read()

# Also available here: https://vl-nat-sec-hackathon-may-2024.s3.us-east-2.amazonaws.com/russia_social_media.csv



In [4]:

import os
import glob
import pandas as pd
import json

# Path to the directory containing files
directory_path = "../extract-entities/files"

# List all json files in the directory
file_list = glob.glob(os.path.join(directory_path, "*.json"))

# Initialize an empty DataFrame to hold all the data
combined_data = []

# Loop through the list of files and read each file into a DataFrame
for file_path in file_list:
    # Read the current json file into a DataFrame
    try:
        with open(file_path) as f:
            data = json.load(f)
        data = [record for record in data if record is not None and isinstance(record, dict)]
        if data != []:
           combined_data.extend(data)

    except:
        print(data)
        print(file_path)
    # Append the data from the current file to the combined DataFrame

# Display the combined DataFrame
# print(combined_data)



In [5]:
print(len(combined_data))

52086


In [36]:
%pip install pandas --break-system-packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 642.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 833.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
# Load the file content into a pandas DataFrame
readdata = pd.read_csv('../../deftech/russia_social_media.csv', nrows=100000)


In [42]:
readdata['time']

0        2024-03-31 18:19:25
1        2024-03-31 07:00:20
2        2024-03-31 09:04:00
3        2024-03-31 06:22:18
4        2024-03-31 17:36:03
                ...         
99995    2024-02-22 06:36:12
99996    2024-02-22 08:23:33
99997    2024-02-22 19:53:31
99998    2024-02-22 07:03:07
99999    2024-02-22 10:53:24
Name: time, Length: 100000, dtype: object

In [32]:
len(readdata['translation'])

480501

In [ ]:
# Write the first 20 rows of the DataFrame to a CSV file
readdata.head(20).to_csv('first_20_rows.csv', index=False)


In [ ]:
res = readdata[readdata['translation'].str.contains('S-300', na=False)]['translation']
import json

# JSON encode the 'res' Series and print
# print(json.dumps(res.head(20).replace("\n", "", regex=True).to_list()))


In [ ]:
[print(x, readdata['translation'][x].replace("\n", "") + "\n\n") for x in range(300, 330) ]


In [ ]:
res = res.head(20)

In [ ]:
[x for x in list(readdata['translation'][:10]) if x != nan]

NameError: name 'nan' is not defined

In [ ]:
len(res['translation'])

TypeError: list indices must be integers or slices, not str

In [18]:
import json
import pandas as pd

# Attempt to open a JSON file and handle potential errors
try:
    with open('o.json', 'r') as file:
        data = json.load(file)
    print("JSON file read successfully")
except FileNotFoundError:
    print("File not found. Please check the file name and path.")
except json.JSONDecodeError:
    print("File is not a valid JSON. Please check the file content.")
except Exception as e:
    print(f"An error occurred: {e}")


JSON file read successfully


In [7]:
data[0][0]

{'event': 'NULL', 'description': 'NULL', 'location': 'NULL'}

x = 

In [7]:
x=combined_data

In [15]:
import math
text_data = [x['description'] for x in combined_data if isinstance(x, dict) ]




# text_data = list(readdata['translation'])
text_data = [x for x in text_data if isinstance(x, str) or not math.isnan(x)]





In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import math

from llm import LLM
model = LLM()
# text_data = res
chunk_size = 1000
embeds = []
for start in range(0, len(text_data), chunk_size):
    end = start + chunk_size
    chunk = text_data[start:end]
    filtered_chunk = [x for x in chunk if isinstance(x, str) or not math.isnan(x)]
    embeds.extend([x.embedding for x in model.embed(filtered_chunk).data])
    

In [17]:
import pickle

# Save the embeddings to a file
with open('embeds.pkl', 'wb') as file:
    pickle.dump(embeds, file)


In [18]:
len(data)

526

In [19]:
import json
text_datda = [x for x in combined_data if isinstance(x, dict) ]

print(len(combined_data))
# text_data = list(readdata['translation'])
text_datda = [x for x in text_datda if isinstance(x['description'], str) or not math.isnan(x['description'])]

# Assuming embeds is a list of embeddings that we want to dump into a JSON file
with open('data.json', 'w') as json_file:
    json.dump(text_datda, json_file)


52086


In [13]:
len(text_datda)

52086

In [1]:
# import pickle

# # Load the embeddings from the file
# with open('embeds.pkl', 'rb') as file:
#     embeddings = pickle.load(file)


In [ ]:
embeddings

In [ ]:
embeddings_list = [embed.embedding for embed in embeds.data]

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

# info = cosine_similarity(embeddings[:30000])
info = cosine_similarity(embeds)


In [13]:
info

array([[1.        , 0.32247615, 0.41079625, ..., 0.1436613 , 0.30849334,
        0.16250105],
       [0.32247615, 1.        , 0.26546352, ..., 0.24482253, 0.29211932,
        0.19271774],
       [0.41079625, 0.26546352, 1.        , ..., 0.18514476, 0.25546436,
        0.09266726],
       ...,
       [0.1436613 , 0.24482253, 0.18514476, ..., 1.        , 0.16343363,
        0.19893062],
       [0.30849334, 0.29211932, 0.25546436, ..., 0.16343363, 1.        ,
        0.20718998],
       [0.16250105, 0.19271774, 0.09266726, ..., 0.19893062, 0.20718998,
        1.        ]])

In [14]:
res = list(res)

NameError: name 'res' is not defined

In [21]:
clustered = {}
used_js = set()
for i, row in enumerate(info):
    clustered[i] = []
    for j, value in enumerate(row):
        if value > 0.6 and i != j and i not in used_js:
            clustered[i].append(j)
            used_js.add(j)


In [27]:
for index, similar_indices in clustered.items():
    if len(similar_indices) > 1:
        print(f"Text at index {index}:")
        print(text_data[index].replace("\n", ""))
        for similar_index in similar_indices:
            print(f"Similar text at index {similar_index}:")
            print(text_data[similar_index].replace("\n", ""))
        print("\n" + "-"*80 + "\n")

Text at index 1:
A crash site of an Il-76 transport plane was cordoned off near the Ivanovo village of Bogorodskoye. The Governor of the Ivanovo region, Stanislav Voskresensky, expressed condolences to the families of the deceased crew pilots and instructed to provide necessary assistance.
Similar text at index 25:
A military Il-76 crashed during takeoff in the Ivanovo region due to an engine fire. The aircraft had eight crew members and seven passengers on board.
Similar text at index 42:
An Il-76 military aircraft crashed during takeoff in the Ivanovo region, resulting in the loss of 15 lives. The cause of the disaster was a fire in one of the engines during takeoff. A commission from the Main Command of the Russian Aerospace Forces was dispatched to investigate the incident.
Similar text at index 115:
A military transport aircraft, Il-76, crashed in the Ivanovo region. Fortunately, there were no casualties or destruction on the ground. Emergency services responded promptly to the in

In [2]:
%pip install scikit-learn --break-system-packages



  Using cached scikit_learn-1.4.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.2-cp312-cp312-macosx_12_0_arm64.whl (10.5 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl (30.4 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pickle

# Load data from data.json file
with open('data.json', 'r') as file:
    data = json.load(file)

# Load embeddings from embed.pkl file
with open('embeds.pkl', 'rb') as file:
    embeddings = pickle.load(file)


In [4]:

import numpy as np

# Assuming 'data' and 'embeddings' are the variables holding the respective data
data_indices = np.arange(len(data))
embedding_indices = np.arange(len(embeddings))

# Ensure reproducibility by setting a seed
np.random.seed(42)

# Randomly sample 20% of the indices from data
sample_size = int(0.2 * len(data_indices))
sampled_data_indices = np.random.choice(data_indices, size=sample_size, replace=False)

# Use the same indices for embeddings to ensure the same elements are sampled
sampled_embedding_indices = sampled_data_indices

# Sample the data and embeddings
sampled_data = [data[i] for i in sampled_data_indices]
sampled_embeddings = [embeddings[i] for i in sampled_embedding_indices]


# Save the sampled data to a new JSON file
with open('data.json', 'w') as file:
    json.dump(sampled_data, file)

# Save the sampled embeddings to a new pickle file
with open('embeds.pkl', 'wb') as file:
    pickle.dump(sampled_embeddings, file)



In [5]:
from cluster import Cluster

cluster = Cluster()
cluster.embed()


In [6]:
cluster.cluster(0.6)
events = [x for x in cluster.get_clusters() if x]

In [7]:
len(events)

10417

In [7]:
import json

# Save the 'events' list to a JSON file named 'events.json'
with open('events.json', 'w') as file:
    json.dump(events, file)


In [23]:
import json
events = json.load(open('events.json'))
print(events)


[[{'event': 'Crash of Il-76 Transport Plane', 'description': 'A crash site of an Il-76 transport plane was cordoned off near the Ivanovo village of Bogorodskoye. The Governor of the Ivanovo region, Stanislav Voskresensky, expressed condolences to the families of the deceased crew pilots and instructed to provide necessary assistance.', 'location': 'Bogorodskoye, Ivanovo Oblast, Russia', 'id': '8db7a969-2bae-5599-bdca-03ca02790a8d', 'time': '2024-03-12 11:35:26', 'original': 'IZ.RUVideo, The crash site of the Il-76 transport plane near the Ivanovo village of Bogorodskoye has been cordoned off.\n\nWork...Governor of the Ivanovo region Stanislav Voskresensky expressed condolences to the families of the deceased crew pilots and instructed to provide all necessary assistanceSubscribe to IZ.RU'}, {'event': 'Il-76 Crash', 'description': 'A military Il-76 crashed during takeoff in the Ivanovo region due to an engine fire. The aircraft had eight crew members and seven passengers on board.', 'lo

In [7]:
# ag_ev =[]
from agent import Agent
agi = Agent()
for event in events[46:]:
    ag_ev.append(agi.extract_event([x['original'] for x in event]))


Agent initialized with LLM and TwitterSearchClient.


In [8]:
len(ag_ev)

54

In [11]:
import json
# Convert 'ag_ev' list of EventLocationResult objects into a list of dictionaries
ag_ev_dicts = [(event[0].dict(), event[1], event[2]) for event in ag_ev]

# Save the 'ag_ev_dicts' list to a JSON file named 'aggregated_events.json'
with open('aggregated_events.json', 'w') as file:
    json.dump(ag_ev_dicts, file)


/var/folders/mz/228hl3dj4yg_nfgk0csb2bs40000gn/T/ipykernel_77209/2667544388.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  ag_ev_dicts = [event.dict() for event in ag_ev]


In [37]:
# Extract the earliest date from each list of events in 'events' and assign it to the corresponding 'ag_ev' object
from datetime import datetime

# Function to parse date from the event dictionary
def parse_date(event):
    date_str = event.get('time', '')
    try:
        return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        return None

# Iterate over 'ag_ev' and 'events' simultaneously and assign the earliest date to 'ag_ev' object
ag_ev_with_min_dates = []
for ag_event, event_group in zip(ag_ev, events):
    if event_group:  # Ensure there are events in the group
        # Extract dates and filter out None values
        dates = filter(None, (parse_date(event) for event in event_group))
        # Extract the 'id' attribute from each event in the event group and add it to a list
        event_ids = [event.get('id') for event in event_group]
        # Find the minimum date and assign it to the 'ag_ev' object if it exists
        date = min(dates, default=None)
    ag_ev_with_min_dates.append((ag_event, date, event_ids))


In [45]:
import json
# Convert 'ag_ev' list of EventLocationResult objects into a list of dictionaries
ag_ev_dicts = [(event[0].dict(), event[1].isoformat(), event[2]) for event in ag_ev_with_min_dates]

# Save the 'ag_ev_dicts' list to a JSON file named 'aggregated_events.json'
with open('aggregated_events.json', 'w') as file:
    json.dump(ag_ev_dicts, file)


/var/folders/mz/228hl3dj4yg_nfgk0csb2bs40000gn/T/ipykernel_77209/1147051201.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  ag_ev_dicts = [(event[0].dict(), event[1].isoformat(), event[2]) for event in ag_ev_with_min_dates]


In [13]:
ag_ev[0].event

'Il-76 military transport plane crashes during takeoff due to engine fire, resulting in no survivors among the 15 onboard'

In [14]:
ag_ev[0].context

['The military Il-76 crashed during takeoff in the Ivanovo region, the Ministry of Defense reports.',
 'There were eight crew members and seven passengers on board.',
 'The cause of the disaster was a fire in one of the engines during takeoff.',
 'A commission from the Main Command of the Russian Aerospace Forces flew to the Ivanovo airfield.',
 'Military correspondents confirm that there are no survivors of the Il-76 military transport plane crash in the Ivanovo region.']

In [18]:
[x['original'] for x in events[0]]



['IZ.RUVideo, The crash site of the Il-76 transport plane near the Ivanovo village of Bogorodskoye has been cordoned off.\n\nWork...Governor of the Ivanovo region Stanislav Voskresensky expressed condolences to the families of the deceased crew pilots and instructed to provide all necessary assistanceSubscribe to IZ.RU',
 'The military Il-76 crashed during takeoff in the Ivanovo region, the Ministry of Defense reports.\n\nThere were eight crew members and seven passengers on board.\n\nThe cause of the disaster was a fire in one of the engines during takeoff. A commission from the Main Command of the Russian Aerospace Forces flew to the Ivanovo airfield.',
 'Ministry of Defense of the Russian Federation: The Il-76 military aircraft crashed during takeoff in the Ivanovo region. There were eight crew members and seven passengers on board.\n\nThe cause of the disaster was a fire in one of the engines during takeoff of the plane. A commission of the Main Command of the Russian Aerospace For